In [9]:
import gzip 
from lxml import etree
import pandas as pd 
import xmltodict
import glob
import os 
import time 
import polars as pl 
import duckdb
import shutil 

def timtim(fonction):
    def wrapper(*args, **kwargs):
        debut = time.time()
        resultat = fonction(*args, **kwargs)
        fin = time.time()
        temps_execution = fin - debut
        print(f"La fonction {fonction.__name__} a pris {temps_execution:.5f} secondes pour s'exécuter.")
        return resultat
    return wrapper

def parse_fichier(chemin) : 
 '''Ouvre et parse le fichier gzip'''
 with gzip.open(chemin, 'rb') as fichier_ouvert : 
  fichier_xml_gzip = fichier_ouvert.read()
  fichier_xml = fichier_xml_gzip.decode('latin-1')
  fichier_dict = xmltodict.parse(fichier_xml)
 return fichier_dict

def recherche_id_fichier(chemin_parquet) :
 conduck = duckdb.connect(database=':memory:', read_only=False)
 docubudg_t = conduck.read_parquet(chemin_parquet)
 requete_duckdb = ''' 
 SELECT
    DISTINCT Id_Fichier
 FROM 
    docubudg_t
 '''
 result_requete= conduck.execute(requete_duckdb).fetchdf()
 liste_id = result_requete['Id_Fichier'].to_list()
 conduck.close()
 return liste_id


def _isolement_id(fichier) : 
 '''Extrait l'id du nom du fichier pour la liste comprehension de securité

 ATTENTION, le premier split / va changer si on l'applique sur du minio '''
 val_id_fichier_source = fichier.split("\\")[-1].split('.')[0]
 if '-' in val_id_fichier_source : 
  val_id_fichier = val_id_fichier_source.split('-')[1]
 else : 
  val_id_fichier= val_id_fichier_source
 return val_id_fichier



Test extraction xml to parquet 

In [19]:
DOSSIER_500 = "./fichiers/todo_xml/test1000/t500/"
DOSSIER_TODO = './fichiers/todo_xml/'
DOSSIER_PARQUET_UNITAIRE = './fichiers/todo_parquet/'

In [15]:
@timtim
def Extraction_xml_to_parquet(chemin) : 
 ''' Traite les XML et les envoies sous format parquet, nettoyage necessaire pour être sous forme de parquet'''
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   for i in docbase : 
      i.update(metadonnees)
   df_methode_2 = pd.DataFrame(docbase)
   df_methode_2['MtSup'] = df_methode_2['MtSup'].astype(str)
   df_methode_2['CaracSup'] = df_methode_2['CaracSup'].astype(str)
   df_methode_2 = df_methode_2.reset_index(drop=True)
   df_methode_2.to_parquet(f'./fichiers/todo_parquet/{id_fichier}', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue  

In [26]:
chemin_test_xml_solo = './fichiers/todo_xml/test1000/t500/616146.xml.gz'


In [21]:
Extraction_xml_to_parquet(DOSSIER_500)

Erreur fichier ./fichiers/todo_xml/test1000/t500\612023.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612058.xml.gz, extraction impossible
'CaracSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612069.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612128.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612136.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612151.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612160.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612180.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612223.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612240.xml.gz, extraction impossible
'MtSup'
Erreur fichier ./fichiers/todo_xml/test1000/t500\612245.x

Très bien, comme d'hab c'est MtSup et CaracSup qui posent problème, ALED. Exploration individuelle 

In [27]:
Extraction_xml_to_parquet(chemin_test_xml_solo)

La fonction Extraction_xml_to_parquet a pris 0.00000 secondes pour s'exécuter.


Ca ne s'execute pas.. on va fouiller
Merci isolement_id qui fait des choses bizarres, dans le doute on va vérifier sur 500 

In [28]:
id_616146 = _isolement_id(chemin_test_xml_solo)
id_616146

''

In [29]:
liste_id_verif_500 = []
for id_test in glob.glob(os.path.join(DOSSIER_500, "*.gz")) : 
 verif_id = _isolement_id(id_test)
 liste_id_verif_500.append(verif_id)

Ici c'est propre. Isolement ne fonctionne donc pas sur du pure unitaire, on va bricoler en attendant c'est pas le coeur du problème

In [30]:
liste_id_verif_500

['612019',
 '612023',
 '612027',
 '612030',
 '612038',
 '612050',
 '612057',
 '612058',
 '612069',
 '612078',
 '612117',
 '612127',
 '612128',
 '612131',
 '612136',
 '612151',
 '612159',
 '612160',
 '612172',
 '612180',
 '612188',
 '612192',
 '612220',
 '612221',
 '612223',
 '612240',
 '612241',
 '612245',
 '612262',
 '612264',
 '612266',
 '612269',
 '612271',
 '612289',
 '612292',
 '612295',
 '612328',
 '612339',
 '612347',
 '612350',
 '612365',
 '612370',
 '612376',
 '612406',
 '612415',
 '612420',
 '612424',
 '612425',
 '612450',
 '612458',
 '612482',
 '612492',
 '612499',
 '612527',
 '612529',
 '612546',
 '612547',
 '612560',
 '612561',
 '612563',
 '612598',
 '612621',
 '612633',
 '612652',
 '612661',
 '612664',
 '612675',
 '612678',
 '612680',
 '612690',
 '612711',
 '612733',
 '612755',
 '612760',
 '612780',
 '612787',
 '612804',
 '612821',
 '612826',
 '612829',
 '612847',
 '612876',
 '612883',
 '612896',
 '612902',
 '612908',
 '612914',
 '612923',
 '612943',
 '612984',
 '612987',

In [31]:
id1_616146 = '616146'

In [32]:
dico_616146 = parse_fichier(chemin_test_xml_solo)
dico_616146

{'DocumentBudgetaire': {'@xmlns': 'http://www.minefi.gouv.fr/cp/demat/docbudgetaire',
  'VersionSchema': {'@V': '88'},
  'BlocEditeur': {'CodeEditeur': {'@V': 'GFI Progiciels, ASTRE GF V7.01 extraction 20141209 procedures 20190617, ASTRE Subventions V7.01'}},
  'VersionOutil': [{'@outil': 'DSC_COMMON', '@version': '27'},
   {'@outil': 'REMAT', '@version': '5_2019'}],
  'Scellement': {'@md5': 'DA5B9388B8CD48E0C112A658DE6F61EC',
   '@sha1': '9DFF3C342C43FDAAADE241774A055FA7D38526FB',
   '@date': '2020-08-06T11:26:34'},
  'EnTeteDocBudgetaire': {'DteStr': {'@V': '2020-03-17'},
   'LibellePoste': {'@V': 'TRESORERIE LYON MUNICIPALE MET'},
   'IdPost': {'@V': '069019'},
   'LibelleColl': {'@V': 'VILLE DE LYON'},
   'IdColl': {'@V': '21690123100011'},
   'NatCEPL': {'@V': 'VILLE'}},
  'Budget': {'EnTeteBudget': {'LibelleEtab': {'@V': 'BUDGET PRINCIPAL'},
    'IdEtab': {'@V': '21690123100011'},
    'CodColl': {'@V': '010'},
    'CodBud': {'@V': '00'},
    'Nomenclature': {'@V': 'M14-M14_COM_SU

In [34]:
metadonnees_616146 = dico_616146['DocumentBudgetaire']['EnTeteDocBudgetaire']
metadonnees_616146['Id_Fichier'] = {'@V' : id1_616146}
metadonnees_616146

{'DteStr': {'@V': '2020-03-17'},
 'LibellePoste': {'@V': 'TRESORERIE LYON MUNICIPALE MET'},
 'IdPost': {'@V': '069019'},
 'LibelleColl': {'@V': 'VILLE DE LYON'},
 'IdColl': {'@V': '21690123100011'},
 'NatCEPL': {'@V': 'VILLE'},
 'Id_Fichier': {'@V': '616146'}}

Bah voilà le problème, c'est sale. 

In [43]:
docbase_616146 = dico_616146['DocumentBudgetaire']['Budget']['LigneBudget']
for i in docbase_616146 : 
 i.update(metadonnees_616146)
df_616146 = pd.DataFrame(docbase_616146)
df_616146.head(3)


,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,{'@V': '6336'},{'@V': '322'},{'@V': '012'},{'@V': 'false'},{'@V': 'D'},{'@V': '0.00'},{'@V': '82638.00'},{'@V': '74680.90'},{'@V': '0'},"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,{'@V': 'VILLE DE LYON'},{'@V': '21690123100011'},{'@V': 'VILLE'},{'@V': '616146'},NaN,NaN,NaN,NaN,NaN,NaN
1,{'@V': '6336'},{'@V': '323'},{'@V': '012'},{'@V': 'false'},{'@V': 'D'},{'@V': '0.00'},{'@V': '12063.00'},{'@V': '8669.70'},{'@V': '0'},"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,{'@V': 'VILLE DE LYON'},{'@V': '21690123100011'},{'@V': 'VILLE'},{'@V': '616146'},NaN,NaN,NaN,NaN,NaN,NaN
2,{'@V': '2313'},{'@V': '213'},{'@V': '23'},{'@V': 'false'},{'@V': 'R'},{'@V': '0.00'},{'@V': '836760.00'},{'@V': '1815.05'},{'@V': '0'},"{'@Code': 'BudgetHorsRAR', '@V': '836760.00'}",...,{'@V': 'VILLE DE LYON'},{'@V': '21690123100011'},{'@V': 'VILLE'},{'@V': '616146'},NaN,NaN,NaN,NaN,NaN,NaN


Il ne faut pas que le nettoyage affecte les colonnes MtSup et CaracSup pour une raison simple : ce ne sera plus des dico, ça ressemblera à rien

In [41]:
def nettoyage_individuel(dataframe : pd.DataFrame) -> pd.DataFrame :
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['MtSup', 'CaracSup'] : 
   continue 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 return dataframe

In [44]:
df_616146_clean = nettoyage_individuel(df_616146)
df_616146_clean.head(5)

,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
2,2313,213,23,false,R,0.00,836760.00,1815.05,0,"{'@Code': 'BudgetHorsRAR', '@V': '836760.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
3,6336,324,012,false,D,0.00,27450.00,25985.93,0,"{'@Code': 'BudgetHorsRAR', '@V': '27450.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
4,6336,33,012,false,D,0.00,418.00,0.00,0,"{'@Code': 'BudgetHorsRAR', '@V': '418.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


C'est le dataframe qu'on veut, maintenant on envoie ça en parquet.

On va aussi tenter avec un autre engine, pour avoir l'erreur sous une autre forme 

In [45]:
df_616146_clean.to_parquet(f'./fichiers/todo_parquet/{id1_616146}', engine='pyarrow')

ArrowInvalid: ('cannot mix list and non-list, non-null values', 'Conversion failed for column MtSup with type object')

Dooooooooooooonc. 

In [46]:
df_616146_clean.to_parquet(f'./fichiers/todo_parquet/{id1_616146}', engine='fastparquet')

On va quand même tenter de nettoyer pour tester sur pyarrow 

In [47]:
df_parquet_616146 = pd.read_parquet('./fichiers/todo_parquet/616146')
df_parquet_616146.head(5)

,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"b'{""@Code"":""BudgetHorsRAR"",""@V"":""82638.00""}'",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"b'{""@Code"":""BudgetHorsRAR"",""@V"":""12063.00""}'",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
2,2313,213,23,false,R,0.00,836760.00,1815.05,0,"b'{""@Code"":""BudgetHorsRAR"",""@V"":""836760.00""}'",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
3,6336,324,012,false,D,0.00,27450.00,25985.93,0,"b'{""@Code"":""BudgetHorsRAR"",""@V"":""27450.00""}'",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
4,6336,33,012,false,D,0.00,418.00,0.00,0,"b'{""@Code"":""BudgetHorsRAR"",""@V"":""418.00""}'",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


In [49]:
df_616146_clean.head(2)

,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


In [50]:
df_616146_pyarrow = df_616146_clean.copy()
df_616146_pyarrow['MtSup'] = df_616146_pyarrow['MtSup'].astype(str)
df_616146_pyarrow.head(2)


,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


In [52]:
df_616146_pyarrow['CaracSup'] = df_616146_pyarrow['CaracSup'].astype(str)
df_616146_pyarrow.head(2)

KeyError: 'CaracSup'

In [55]:
df_616146_pyarrow.columns

Index(['Nature', 'Fonction', 'ContNat', 'ArtSpe', 'CodRD', 'MtPrev', 'CredOuv',
       'MtReal', 'OpBudg', 'MtSup', 'DteStr', 'LibellePoste', 'IdPost',
       'LibelleColl', 'IdColl', 'NatCEPL', 'Id_Fichier', 'MtRARPrec',
       'TypOpBudg', 'MtRAR3112', 'LibCpte', 'ContOp', 'OpeCpteTiers'],
      dtype='object')

Il n'y a pas de caracsup, c'est le défaut de faire des test sur des fichiers seuls. On va juste vérifier que le fichier n'ait pas disparu pendant le chemin 

In [56]:
df_616146.columns

Index(['Nature', 'Fonction', 'ContNat', 'ArtSpe', 'CodRD', 'MtPrev', 'CredOuv',
       'MtReal', 'OpBudg', 'MtSup', 'DteStr', 'LibellePoste', 'IdPost',
       'LibelleColl', 'IdColl', 'NatCEPL', 'Id_Fichier', 'MtRARPrec',
       'TypOpBudg', 'MtRAR3112', 'LibCpte', 'ContOp', 'OpeCpteTiers'],
      dtype='object')

In [57]:
df_616146_pyarrow = df_616146_pyarrow.reset_index(drop=True)
df_616146_pyarrow.head(3)

,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
2,2313,213,23,false,R,0.00,836760.00,1815.05,0,"{'@Code': 'BudgetHorsRAR', '@V': '836760.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


In [58]:
df_616146_pyarrow.to_parquet(f'./fichiers/todo_parquet/{id1_616146}_pyarrow', engine='pyarrow')

In [59]:
df_parquet_arrow = pd.read_parquet(f'./fichiers/todo_parquet/616146_pyarrow')
df_parquet_arrow.head(2)

,Nature,Fonction,ContNat,ArtSpe,CodRD,MtPrev,CredOuv,MtReal,OpBudg,MtSup,...,LibelleColl,IdColl,NatCEPL,Id_Fichier,MtRARPrec,TypOpBudg,MtRAR3112,LibCpte,ContOp,OpeCpteTiers
0,6336,322,012,false,D,0.00,82638.00,74680.90,0,"{'@Code': 'BudgetHorsRAR', '@V': '82638.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan
1,6336,323,012,false,D,0.00,12063.00,8669.70,0,"{'@Code': 'BudgetHorsRAR', '@V': '12063.00'}",...,VILLE DE LYON,21690123100011,VILLE,616146,nan,nan,nan,nan,nan,nan


Pyarrow est bien plus rapide. On va donc rester dessus et créer une fonction de nettoyage qui s'adapte à l'existence de CaracSup

In [60]:
def nettoyage_adaptatif(dataframe : pd.DataFrame) -> pd.DataFrame :
 nettoyage = lambda x : str(x).replace("{'@V': '", "").replace("'}", "")
 for col in dataframe.columns : 
  if col in ['MtSup', 'CaracSup'] : 
   dataframe[col] = dataframe[col].astype(str) 
  else :
   dataframe[col] = dataframe[col].apply(nettoyage)
 dataframe_propre = dataframe.reset_index(drop=True)
 return dataframe_propre

On va refaire un test avec ce nouveau nettoyage sur les 500 fichiers 

In [62]:
def Extraction_xml_to_parquet_v2(chemin) : 
 ''' Traite les XML et les envoies sous format parquet, nettoyage necessaire pour être sous forme de parquet'''
 for fichier in glob.glob(os.path.join(chemin, "*.gz")) : 
  try : 
   id_fichier = _isolement_id(fichier)
   dico = parse_fichier(fichier)
   metadonnees = dico['DocumentBudgetaire']['EnTeteDocBudgetaire']
   metadonnees['Id_Fichier'] = {'@V' : id_fichier}
   docbase = dico['DocumentBudgetaire']['Budget']['LigneBudget']
   for i in docbase : 
      i.update(metadonnees)
   df_base = pd.DataFrame(docbase)
   df_propre = nettoyage_adaptatif(df_base)
   df_propre.to_parquet(f'./fichiers/todo_parquet/{id_fichier}_pyarrow', engine='pyarrow')
   #shutil.move(fichier, './fichiers/done_xml/')
  except Exception as e : 
    print(f'Erreur fichier {fichier}, extraction impossible')
    #shutil.move(fichier, './fichiers/todo_xml/error/')
    print(e)
    continue  

In [63]:
Extraction_xml_to_parquet_v2(DOSSIER_500)

Victoire, sûrement de courte durée. En tout cas on a la task 1 pour airflow, peut être. 

In [64]:
df_numero_osef = pd.read_parquet('./fichiers/todo_parquet/612027_pyarrow')
df_numero_osef.head(2)

,Nature,ContNat,ArtSpe,CodRD,MtBudgPrec,MtPropNouv,MtPrev,CredOuv,OpBudg,MtSup,CaracSup,DteStr,LibellePoste,IdPost,LibelleColl,IdColl,NatCEPL,Departement,Id_Fichier
0,6232,011,false,D,200.00,850.00,850.00,850.00,0,"{'@V': '200.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",2020-08-01,Trésorerie de BEAUFORT,039005,Commune de AUGEA,26390025000014,COMMUNE,39,612027
1,6288,011,false,D,2597.00,360.61,360.61,360.61,0,"{'@V': '2597.00', '@Code': 'BudgetHorsRAR'}","{'@V': 'F', '@Code': 'Section'}",2020-08-01,Trésorerie de BEAUFORT,039005,Commune de AUGEA,26390025000014,COMMUNE,39,612027
